In [3]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchmetrics
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import lightning as L

In [2]:
offers = []
for root, _, files in os.walk(f"../images"):
    if len(root.split("/")) == 5:
        offers.append("/".join(root.split("/")[2:]))

offers

['30805/3685/OMS',
 '22508/3685/OMS',
 '27080/3685/OMS',
 '30832/3685/OMS',
 '27285/3685/OMS',
 '27282/3685/OMS',
 '22101/3685/OMS',
 '25924/3685/OMS',
 '29208/3685/OMS',
 '24977/3685/OMS',
 '31096/3685/OMS',
 '28987/3685/OMS',
 '24127/3685/OMS',
 '30778/3685/OMS',
 '22474/3685/OMS',
 '29044/3685/OMS',
 '25484/3685/OMS',
 '29623/3685/OMS',
 '22831/3685/OMS',
 '31211/3685/OMS',
 '28767/3685/OMS',
 '24502/3685/OMS',
 '29614/3685/OMS',
 '29045/3685/OMS',
 '31288/3685/OMS',
 '28902/3685/OMS',
 '29358/3685/OMS',
 '29361/3685/OMS',
 '31399/3685/OMS',
 '23238/3685/OMS',
 '27495/3685/OMS',
 '31133/3685/OMS',
 '27440/3685/OMS',
 '27827/3685/OMS',
 '27280/3685/OMS',
 '25147/3685/OMS',
 '27226/3685/OMS',
 '31344/3685/OMS',
 '28090/3685/OMS',
 '23471/3685/OMS',
 '29127/3685/OMS',
 '25389/3685/OMS',
 '30831/3685/OMS',
 '24123/3685/OMS',
 '26618/3685/OMS',
 '28111/3685/OMS',
 '24780/3685/OMS',
 '28516/3685/OMS',
 '25664/3685/OMS',
 '26218/3685/OMS',
 '29031/3685/OMS',
 '23159/3685/OMS',
 '24981/3685

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [4]:
def get_photos(offer_id, transform=None):
    batch = []
    for root, _, files in os.walk(f"../images/{offer_id}"):
        for file in files:
            if file.endswith(".jpg") and "all" not in file and "wp" not in file:
                image_path = os.path.join(root, file)
                image = Image.open(image_path).convert("RGB")

                if transform:
                    image = transform(image)
                else:
                    image = transforms.ToTensor()(image)

                batch.append(image)

    return torch.stack(batch)

In [5]:
s = 14 * 16
m = 14 * 32 
l = 14 * 48 
xl = 14 * 64

S = (s, s)
M = (m, m)
L = (l, l) # good for 18GB VRAM
XL = (xl, xl)

transform = transforms.Compose([
    transforms.Resize(L),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

model = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14_reg").to(device)

Using cache found in /Users/kacper/.cache/torch/hub/facebookresearch_dinov2_main
/Users/kacper/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/Users/kacper/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/Users/kacper/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [6]:
for offer in offers:
    print(f"Processing {offer}")

    images = get_photos(offer_id=offer, transform=transform)
    print(images.shape)

    outputs = []
    for i, image in tqdm(enumerate(images), total=len(images)):
        image = image.unsqueeze(0).to(device)
        output = model(image).squeeze(0).detach().cpu().numpy()
        outputs.append(output)

    outputs = np.array(outputs)
    filename = "_".join(offer.split("/"))
    with open(f'../images_embeded/{filename}.pkl', 'wb') as f:
        pickle.dump({
            'image_size': L,
            'offer_id': offer,
            'outputs': outputs,
        }, f)
    print(f"Saved to ../images_embeded/{filename}.pkl")

Processing 30805/3685/OMS
torch.Size([28, 3, 672, 672])


  0%|          | 0/28 [00:00<?, ?it/s]/opt/miniconda3/envs/real-estate-dinov2-test/lib/python3.11/site-packages/torch/nn/functional.py:4072: UserWarning: The operator 'aten::_upsample_bicubic2d_aa.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1716905753263/work/aten/src/ATen/mps/MPSFallback.mm:13.)
  return torch._C._nn._upsample_bicubic2d_aa(input, output_size, align_corners, scale_factors)
100%|██████████| 28/28 [00:03<00:00,  8.76it/s]


Saved to ../images_embeded/30805_3685_OMS.pkl
Processing 22508/3685/OMS
torch.Size([20, 3, 672, 672])


100%|██████████| 20/20 [00:02<00:00,  9.31it/s]


Saved to ../images_embeded/22508_3685_OMS.pkl
Processing 27080/3685/OMS
torch.Size([20, 3, 672, 672])


...

Saved to ../images_embeded/21357_3685_OMS.pkl
Processing 26529/3685/OMS
torch.Size([18, 3, 672, 672])


100%|██████████| 18/18 [00:01<00:00,  9.26it/s]


Saved to ../images_embeded/26529_3685_OMS.pkl
Processing 26784/3685/OMS
torch.Size([15, 3, 672, 672])


100%|██████████| 15/15 [00:01<00:00,  9.14it/s]

Saved to ../images_embeded/26784_3685_OMS.pkl


In [6]:
with open("../images_embeded/30805_3685_OMS.pkl", "rb") as f:
    data = pickle.load(f)
data

{'image_size': (672, 672),
 'offer_id': '30805/3685/OMS',
 'outputs': array([[ 2.560301  , -0.8504364 , -0.21759939, ...,  1.52431   ,
          1.125218  ,  1.6827465 ],
        [ 1.9245269 , -0.48709422,  1.9455004 , ..., -0.94634795,
          0.5189094 ,  0.967928  ],
        [ 1.4143376 , -1.6160402 ,  0.3519848 , ...,  2.9981334 ,
          1.7953353 ,  2.053301  ],
        ...,
        [-0.58763826, -0.72419184, -0.693054  , ..., -1.3873552 ,
          1.4864492 , -2.251434  ],
        [ 1.7955742 , -1.2687701 , -0.7398259 , ..., -0.5495759 ,
          1.591101  ,  0.20677093],
        [ 1.497448  , -0.86951196,  0.15145272, ..., -0.02245994,
          1.4777577 ,  1.5281867 ]], dtype=float32)}